In [1]:
import os

In [2]:
%pwd

'd:\\DS\\Projects\\DL\\Cars-Tanks-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\DS\\Projects\\DL\\Cars-Tanks-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

        
    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, self.config.data_ingestion.dataset_name)
        
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE
        )

        return training_config

In [8]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1.0 / 255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bicubic"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                brightness_range=(0.8, 1.2),
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        # Define the ModelCheckpoint callback
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.trained_model_path,  # Save the best model here
            monitor="val_loss",                      # Monitor validation loss
            save_best_only=True,                     # Save only the best model
            mode="min",                              # Minimize the validation loss
            verbose=1                                # Print a message when saving
        )

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator,
            callbacks=[checkpoint_callback]  # Include the callback
        )


In [9]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2025-01-05 22:39:39,870: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-05 22:39:39,876: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-05 22:39:39,878: INFO: common: created directory at: artifacts]
[2025-01-05 22:39:39,881: INFO: common: created directory at: artifacts\training]
Found 300 images belonging to 2 classes.
Found 1203 images belonging to 2 classes.
Epoch 1/10
75/75 [==============================] - ETA: 0s - loss: 0.4418 - accuracy: 0.9680 - precision: 0.9680 - recall: 0.9680
Epoch 1: val_loss improved from inf to 0.49708, saving model to artifacts\training\model.keras
75/75 [==============================] - 105s 1s/step - loss: 0.4418 - accuracy: 0.9680 - precision: 0.9680 - recall: 0.9680 - val_loss: 0.4971 - val_accuracy: 0.9931 - val_precision: 0.9931 - val_recall: 0.9931
Epoch 2/10
75/75 [==============================] - ETA: 0s - loss: 0.0977 - accuracy: 0.9941 - precision: 0.9941 - recall: 0.9941
Epoch 2: val_loss 